In [1]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [ ]:
print(stores_train.shape)
stores_train.head()

In [ ]:
print(stores_val.shape)
stores_val.head()

In [ ]:
print(stores_test.shape)
stores_test.head()

In [ ]:
sns.histplot(data=stores_train['revenue'], bins=50)

### Investigating the different features

In [ ]:
stores_train.dtypes

In [ ]:
stores_train.describe()

store_id and year are redundant as they provide no information

In [ ]:
stores_train["store_name"].nunique()


Since there are so many unique store names, we omit this ATM

#### Plaace Hierarchy ID

In [ ]:
stores_train["plaace_hierarchy_id"].value_counts()

In [ ]:
stores_train["plaace_hierarchy_id_6"] = stores_train["plaace_hierarchy_id"].apply(lambda x: x[:3])
stores_train["plaace_hierarchy_id_6"].value_counts()

IDEA: split into 4 columns, first column contains only first number, second contains first two numbers...

IDEA: split the 4 numbers into 4 columns.

Treat as categorical variable

#### Sales Channel Name

In [ ]:
stores_train["sales_channel_name"].value_counts()

Contains same information as plaace hierarchy id, redundant

#### Grunnkrets ID

In [ ]:
stores_train["grunnkrets_id"]

Simply a foreign key to link to the other CSV files, will look at it later

#### Address

In [ ]:
stores_train["address"].nunique()

#### Lat & Long

In [ ]:
plt.plot(stores_train["lon"], stores_train["lat"], "bo")
plt.ylabel("lon")
plt.xlabel("lat")
plt.show();

#### Chain Name

In [ ]:
# pd.set_option('display.max_rows', None)  # or 1000
stores_train["chain_name"].value_counts()

In [ ]:
stores_train["chain_name"].isna().sum() / stores_train.shape[0]

IMPUTE NANS: impute NANs as a category of its own. Can also treat the whole column as binary not-NAN/NAN. Can also decide threshold for when a chain becomes a NAN or another category altogether.

#### Mall Name

In [ ]:
stores_train["mall_name"].isna().sum() / stores_train.shape[0]

In [ ]:
pd.set_option('display.max_rows', 500)
stores_train["mall_name"].value_counts()

Treat similarly to chain name, we think a binary approach would be the best as the size of a lot of malls seems to be wrong due to missing shops etc.

#### Revenue

In [ ]:
stores_train.revenue.plot.hist(bins=50, logy=True);

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 3), ncols=3)
stores_train.isna().mean().plot.bar(ax=ax1)
ax1.set_title('Fraction of rows with NaN values (train)')
stores_test.isna().mean().plot.bar(ax=ax2)
ax2.set_title('Fraction of rows with NaN values (test)')
stores_train.revenue.plot.hist(bins=100, ax=ax3)
ax3.set_title('Distribution of Revenues');

### Feature engineering

We start by including lat, lon, chain_name, mall_name and plaace_hierarchy_id

In [ ]:
stores_train["is_mall"] = ~stores_train["mall_name"].isna()

In [ ]:
lower_limit = 10

chain_count = stores_train["chain_name"].value_counts().to_dict()
stores_train["bounded_chain_name"] = stores_train["chain_name"].apply(lambda x: "OTHER" if(x in chain_count and chain_count[x] < lower_limit) else x)
stores_train[["chain_name", "bounded_chain_name"]].head()

In [ ]:
stores_train["plaace_cat_1"] = stores_train["plaace_hierarchy_id"].apply(lambda x: x[:1])
stores_train["plaace_cat_2"] = stores_train["plaace_hierarchy_id"].apply(lambda x: x[:3])
stores_train["plaace_cat_3"] = stores_train["plaace_hierarchy_id"].apply(lambda x: x[:5])
stores_train["plaace_cat_4"] = stores_train["plaace_hierarchy_id"]
stores_train[["plaace_cat_1", "plaace_cat_2", "plaace_cat_3", "plaace_cat_4"]].head()

In [ ]:
stores_train["plaace_cat_4"].unique()

In [ ]:
stores_train[stores_train["plaace_cat_1"] == "1"].shape

In [ ]:
unique_vals = stores_train["plaace_cat_" + str(1)].unique()
unique_vals

In [ ]:
for val in unique_vals:
    filtered_df = stores_train[stores_train["plaace_cat_1"] == val]
    filtered_df["random"] = 1

In [ ]:
from scipy.spatial.distance import cdist


def closest_point(point, points):
    """ Find closest point from a list of points. """
    if(len(points) == 0):
        return None
    return points[cdist([point], points).argmin()]

**!NB**

Next cell can take up to 1 minute to run

In [ ]:
stores_train["point"] = [(x, y) for x,y in zip(stores_train['lat'], stores_train['lon'])]
stores_train['closest'] = [closest_point(x["point"], list(stores_train.loc[stores_train["plaace_cat_3"] == x["plaace_cat_3"]]['point'].drop([i], axis=0))) for i, x in stores_train.iterrows()]

In [ ]:
stores_train["closest"].isna().sum()

In [ ]:
stores_train["closest"].head()

In [ ]:
for i, row in stores_train.iterrows():
    if(row["closest"] == None):
        val = float("inf")
    else:
        val = cdist(np.array(row["point"]).reshape(1, -1), np.array(row["closest"]).reshape(1, -1))
    stores_train.at[i,'dist_to_nearest_comp'] = val

In [ ]:
stores_train.sample(5)

In [ ]:
from utils import create_geographical_columns


stores_train = create_geographical_columns(stores_train)

In [ ]:
unq_vals = stores_train["plaace_cat_4"].unique()

In [ ]:
rev_dict = {}
for val in unq_vals:
    rev_dict[val] = stores_train["revenue"].where(stores_train["plaace_cat_4"] == val).mean()

In [ ]:
stores_train["mean_revenue"] = stores_train["plaace_cat_4"].apply(lambda x: rev_dict[x])

In [ ]:
chain_count = stores_train["chain_name"].value_counts().to_dict()
chain_count[np.nan] = 0
lower_limit = 10

rev_dict = {}
mean_revenue = stores_train.revenue.mean()
for val in unq_vals:
    rev_dict[val] = stores_train["revenue"].where(stores_train["plaace_cat_4"] == val).mean()

def generate_rev_dict(df, plaace_cat_granularity: int = 4):
    rev_dict = {}
    mean_revenue = df.revenue.mean()
    for val in df["plaace_cat_" + str(plaace_cat_granularity)]:
        rev_dict[val] = df["revenue"].where(df["plaace_cat_" + str(plaace_cat_granularity)] == val).mean()
    return rev_dict, mean_revenue

def mean_func_rev(plaace_cat, rev_dict, mean_revenue):
    if(plaace_cat in rev_dict.keys()):
        return rev_dict[plaace_cat]
    return mean_revenue

def feature_engineer_df(
    df: pd.DataFrame, 
    chain_count: dict, 
    rev_dict: dict, 
    training: bool = True, 
    training_df: pd.DataFrame = None, 
    lower_limit: int = 10, 
    plaace_cat_granularity: int = 4
):
    df["is_mall"] = ~df["mall_name"].isna()
    df["is_chain"] = ~df["chain_name"].isna()
    df["bounded_chain_name"] = df["chain_name"].apply(lambda x: "OTHER" if(x in chain_count and chain_count[x] < lower_limit) else x)
    df["plaace_cat_1"] = df["plaace_hierarchy_id"].apply(lambda x: x[:1])
    df["plaace_cat_2"] = df["plaace_hierarchy_id"].apply(lambda x: x[:3])
    df["plaace_cat_3"] = df["plaace_hierarchy_id"].apply(lambda x: x[:5])
    df["plaace_cat_4"] = df["plaace_hierarchy_id"]
    df["point"] = [(x, y) for x,y in zip(df['lat'], df['lon'])]
    training_df["point"] = [(x, y) for x,y in zip(training_df['lat'], training_df['lon'])]
    if training:
        df['closest_' + str(plaace_cat_granularity)] = [
            closest_point(
                x["point"], 
                list(training_df.loc[
                    training_df["plaace_cat_" + str(plaace_cat_granularity)] == x["plaace_cat_" + str(plaace_cat_granularity)]
                    ]['point'].drop([i], axis=0))) for i, x in df.iterrows()
            ]
    else:
        df['closest_' + str(plaace_cat_granularity)] = [
            closest_point(
                x["point"], 
                list(training_df.loc[
                    training_df["plaace_cat_" + str(plaace_cat_granularity)] == x["plaace_cat_" + str(plaace_cat_granularity)]
                    ]['point'])) for i, x in df.iterrows()
            ]
    df["mean_revenue_" + str(plaace_cat_granularity)] = df["plaace_cat_" + str(plaace_cat_granularity)].apply(lambda x: mean_func_rev(x, rev_dict, mean_revenue))
    for i, row in df.iterrows():
        if(row["closest_" + str(plaace_cat_granularity)] == None):
            val = np.nan
        else:
            val = cdist(np.array(row["point"]).reshape(1, -1), np.array(row["closest_" + str(plaace_cat_granularity)]).reshape(1, -1))
        df.at[i,'dist_to_nearest_comp'] = val
    df = create_geographical_columns(df)
    return df
    

In [ ]:
stores_extra = pd.read_csv("data/stores_extra.csv")
stores_extra.index += stores_train.index.max()

In [ ]:
concat_df = pd.concat([stores_train, stores_extra])

In [ ]:
for i in range(1, 5):
    stores_train = feature_engineer_df(stores_train, chain_count, rev_dict, training_df=concat_df, plaace_cat_granularity=i)
    stores_val = feature_engineer_df(stores_val, chain_count, rev_dict, training=False, training_df=concat_df, plaace_cat_granularity=i)
    stores_test = feature_engineer_df(stores_test, chain_count, rev_dict, training=False, training_df=concat_df, plaace_cat_granularity=i)

In [ ]:
stores_train.dist_to_nearest_comp.describe()

In [ ]:
stores_train.dist_to_nearest_comp.describe()

## Preprocessing data & training model

We start by including lat, lon, chain_name, mall_name and plaace_hierarchy_id

In [ ]:
from utils import preprocess_grunnkrets_df, create_geographical_columns

class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self


In [ ]:
full_population_df = pd.read_csv("temp_data/full_population_data_train.csv")
closest_bus_stop_df = pd.read_csv("temp_data/closest_bus_stops_train.csv")

full_stores_train = stores_train.merge(full_population_df, left_on="store_id", right_on="store_id")
full_stores_train = full_stores_train.merge(closest_bus_stop_df, left_on="store_id", right_on="store_id")

In [ ]:
full_stores_val = stores_val.merge(full_population_df, left_on="store_id", right_on="store_id")
full_stores_val = full_stores_val.merge(closest_bus_stop_df, left_on="store_id", right_on="store_id")

In [ ]:
full_population_df_test = pd.read_csv("temp_data/full_population_data_test.csv")
closest_bus_stop_df_test = pd.read_csv("temp_data/closest_bus_stops_test.csv")

In [ ]:
full_stores_test = stores_test.merge(full_population_df_test, left_on="store_id", right_on="store_id")
full_stores_test = full_stores_test.merge(closest_bus_stop_df_test, left_on="store_id", right_on="store_id")

In [ ]:
fylke_relevant_features = [col_name for col_name in full_stores_train.columns if col_name.startswith("fylke.")]
kommune_relevant_features = [col_name for col_name in full_stores_train.columns if col_name.startswith("kommune.")]
delomrade_relevant_features = [col_name for col_name in full_stores_train.columns if col_name.startswith("delomrade.")]
grunnkrets_relevant_features = [col_name for col_name in full_stores_train.columns if col_name.startswith("grunnkrets_id.")]

In [ ]:
log_delta = 1
full_stores_train["log_revenue"] = full_stores_train.revenue.apply(lambda x: np.log(x + log_delta))

In [ ]:
grocery_removed_train = full_stores_train[full_stores_train["sales_channel_name"] != "Grocery stores"]
grocery_removed_val = full_stores_val[full_stores_val["sales_channel_name"] != "Grocery stores"]

In [ ]:
grocery_removed_train["log_revenue"] = grocery_removed_train.revenue.apply(lambda x: np.log(x + log_delta))

In [ ]:
grocery_removed_train.shape

In [ ]:
full_stores_train["is_grocery"] = full_stores_train.sales_channel_name.apply(lambda x: x == "Grocery stores")
full_stores_val["is_grocery"] = full_stores_val.sales_channel_name.apply(lambda x: x == "Grocery stores")

In [ ]:
full_stores_val.is_grocery.sum()

In [2]:
stores_train = pd.read_csv("temp_data/full_features_train.csv", index_col=0)
stores_val = pd.read_csv("temp_data/full_features_val.csv", index_col=0)
stores_extra = pd.read_csv("temp_data/full_features_extra.csv", index_col=0)
stores_test = pd.read_csv("temp_data/full_features_test.csv", index_col=0)

store_dataframes = {
    "train": stores_train, 
    "extra": stores_extra, 
    "test": stores_test, 
    "val": stores_val
    }

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


OE_categorical_features = ["bounded_chain_name", "kommune", "delomrade", "is_grocery", "plaace_cat_3", "plaace_cat_4"]
OE_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

OH_categorical_features = ["fylke", "plaace_cat_2"]
OH_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)


numerical_features = ["lat", "lon", "dist_to_nearest_comp", 
"mean_revenue_1", "mean_revenue_2", "mean_revenue_3", "mean_revenue_4", 
] + delomrade_relevant_features + list(closest_bus_stop_df.columns[1:])
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")), 
        ("scaler", StandardScaler(with_mean=True, with_std=True))]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)


X_train = preprocessor.fit_transform(full_stores_train)
X_val = preprocessor.transform(full_stores_val)

In [ ]:
y_train = np.array(full_stores_train.log_revenue)
y_val = np.array(full_stores_val.revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

In [ ]:
X_train.shape

# Models

In [ ]:
from sklearn.model_selection import GridSearchCV
from RMSLE import rmsle

## Simple Models

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=250, max_features=8, min_samples_leaf=2, min_samples_split=16)
rf_params = {
    "n_estimators" : (100, 250, 500, 1000), 
    "max_features" : (1, 2, 4, 8), 
    "min_samples_split" : (4, 8, 16, 32), 
    "min_samples_leaf" : (2, 4, 8), 
    }

rf_clf = GridSearchCV(rf, rf_params, verbose=2)

!NB Next cell takes several minutes to run (~5 minutes)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_clf.best_params_

In [ ]:
rf_y_pred = rf.predict(X_val)
rmsle(y_pred=rf_y_pred, y_true=y_val)

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_clf = LinearRegression(n_jobs=-1)

In [ ]:
lr_clf.fit(X_train, y_train)

In [ ]:
lr_y_pred = lr_clf.predict(X_val)

In [ ]:
lr_y_pred = np.array([max(0, xi) for xi in lr_y_pred])

In [ ]:
rmsle(y_pred=lr_y_pred, y_true=y_val)

### Light GBM

In [ ]:
from lightgbm import LGBMRegressor

# Current best params
lgbm = LGBMRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, n_estimators=100, reg_lambda=0.01)
lgbm_params = {
    #"num_leaves" : (10, 25, 31, 75), 
    "learning_rate" : (0.05, 0.1, 0.25), 
    "n_estimators" : (50, 100, 250), 
    #"min_split_gain" : (0, 0.01, 0.1), 
    #"min_child_samples" : (4, 8, 16, 32), 
    "reg_alpha" : (0, 0.01, 0.1), 
    "reg_lambda" : (0, 0.01, 0.1), 
    }

lgbm_clf = GridSearchCV(lgbm, lgbm_params)

!NB Depending on the total possible configurations of hyperparams, the next cell can take veeeeery long 

In [ ]:
lgbm.fit(X_train, y_train)

In [ ]:
lgbm_clf.best_params_

In [ ]:
lgbm_y_pred = lgbm.predict(X_val)

In [ ]:
lgbm_y_pred = np.array([max(0, xi) for xi in lgbm_y_pred])

In [ ]:
rmsle(y_pred=lgbm_y_pred, y_true=y_val)

## Stacking classifiers

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', rf), 
    ('lf', lr_clf), 
    ('lgbm', lgbm), 
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=50, random_state=0, n_jobs=-1)
)

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
reg_y_pred = reg.predict(X_val)
reg_y_pred *= std_y
reg_y_pred += mean_y
reg_y_pred = np.exp(reg_y_pred) - log_delta

In [ ]:
rmsle(y_pred=reg_y_pred, y_true=y_val)

## See largest contributors to high RMSLE

In [ ]:
pred_diff = np.abs(reg_y_pred - y_val)

In [ ]:
n_largest_diff = 100
n_largest_index = np.argsort(-pred_diff)[:n_largest_diff]

In [ ]:
grocery_index = list(full_stores_val.index[full_stores_val["sales_channel_name"] == "Grocery stores"])

In [ ]:
grocery_removed_y_pred = np.delete(reg_y_pred, grocery_index)

In [ ]:
grocery_removed_y_val = np.delete(y_val, grocery_index)

In [ ]:
n_largest_wrong_df = full_stores_val.iloc[n_largest_index]

In [ ]:
n_largest_wrong_df.sales_channel_name.value_counts()

# Creating the submission

In [ ]:
# Predict on the test set 
X_test = preprocessor.transform(full_stores_test)
y_test_pred = reg.predict(X_test)
y_test_pred *= std_y
y_test_pred += mean_y
y_test_pred = np.exp(y_test_pred) - log_delta

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)